<a href="https://colab.research.google.com/github/saathvikMD/actor_critic/blob/main/lunar_lander_reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#          _____                _____                    _____                    _____                    _____                    _____          
#         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
#        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
#       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
#      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
#     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
#    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
#    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
#  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
# /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
#/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
#\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
# \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
#  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
#   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
#    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
#     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
#      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
#       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
#        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
#         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/    

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (51.1.1)


In [ ]:
!pip install box2d-py
!pip install gym[Box_2D]

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

import numpy as np
import torch
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video(folder):
  mp4list = glob.glob(folder + '/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env, folder):
  env = Monitor(env, './' + folder, force = True)
  return env

In [ ]:
def add_zeros(x, max):
  max_len = len(str(max))
  if len(str(x)) == max_len - 0:
    return str(x)
  elif len(str(x)) == max_len - 1:
    return '0' + str(x)
  elif len(str(x)) == max_len - 2:
    return '00' + str(x)
  elif len(str(x)) == max_len - 3:
    return '000' + str(x)

In [ ]:
class Agent():
  def __init__(self, lr, gamma, input_dims, actions):
    self.lr = lr
    self.gamma = gamma
    self.reward_memory = []
    self.action_memory = []
    self.model = torch.nn.Sequential(*[torch.nn.Linear(input_dims, input_dims * 2), torch.nn.Linear(input_dims * 2, input_dims), torch.nn.Linear(input_dims, actions), torch.nn.Softmax()])
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr = lr)
  
  def forward(self, x):
    return self.model(x)
  
  def choose_action(self, state):
    state = torch.Tensor(state)
    probabilities = self.forward(state)
    action_probs = torch.distributions.Categorical(probabilities)
    action = action_probs.sample()
    log_probs = action_probs.log_prob(action)
    self.action_memory.append(log_probs)

    return action.item()

  def learn(self):
    self.optimizer.zero_grad()
    G = np.zeros_like(self.reward_memory, dtype = np.float64)
    for t in range(len(self.reward_memory)):
      G_sum = 0
      discount = 1
      for k in range(t, len(self.reward_memory)):
        G_sum += self.reward_memory[k] * discount
        discount *= self.gamma
      G[t] = G_sum
    G = torch.tensor(G, dtype = torch.float)

    loss = 0
    for g, log_prob in zip(G, self.action_memory):
      loss += -g * log_prob
    loss.backward()
    self.optimizer.step()

    self.action_memory = []
    self.reward_memory = []
  
  def store_rewards(self, reward):
    self.reward_memory.append(reward)

In [ ]:
def plot_learning_curve(scores, filename, length = 100):
  running_avg = [np.mean(scores[max(0, i-length):(i + 1)]) for i in range(len(csores))]
  x = np.zeros(len(scores))
  plt.plot(x, running_avg)
  plt.title('average of previous ' + str(length))
  plt.savefig(filname)

In [ ]:
env = wrap_env(gym.make('LunarLander-v2'), 'video')
state = env.reset()
scores = []
n_games = 5000
agent = Agent(0.0005, 0.99, len(state), env.action_space.n)
for episode in range(n_games):
  env = wrap_env(gym.make('LunarLander-v2'), 'video' + str(episode))
  current_state = env.reset()
  game_over = False
  score = 0
  while not game_over:
    action = agent.choose_action(current_state)
    next_state, reward, game_over, info = env.step(action)
    agent.store_rewards(reward)
    score += reward
    current_state = next_state
  agent.learn()
  scores.append(score)
  avg_score = np.mean(scores[-100:])
  print('episode :', str(episode), ' score: ', str(int(score)), ' avg_score: ' + str(int(avg_score)))


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


episode : 0  score:  -197  avg_score: -197


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


episode : 1  score:  -99  avg_score: -148


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


...

episode : 2173  score:  -79  avg_score: -124


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


episode : 2174  score:  -109  avg_score: -124


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


KeyboardInterrupt: ignored

In [ ]:
env = wrap_env(gym.make('LunarLander-v2'), 'video_Test')
state= env.reset()
current_state = env.reset()
game_over = False
score = 0
test_agent = Agent(0.0005, 0.99, len(state), env.action_space.n)
while not game_over:
  action = env.action_space.sample()
  next_state, reward, game_over, info = env.step(action)
  test_agent.store_rewards(reward)
  score += reward
  current_state = next_state
print(score)
show_video('video_Test')

-247.19718553083777
